In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import scipy.signal as signal
import scipy.fftpack as fftpack
import timeit, glob
from scipy.signal import butter, lfilter, sosfiltfilt
from matplotlib import rcParams
from matplotlib import pyplot as plt

In [2]:
# Load the big data file downloaded from https://figshare.com/articles/dataset/LFP_data_from_Steinmetz_et_al_2019/9727895
rootPath = "/run/media/h4wke/Elements/LFP/"

Sessions = ['Cori_2016-12-14', 'Cori_2016-12-17', 'Cori_2016-12-18', 'Forssmann_2017-11-01', 'Forssmann_2017-11-02', 'Forssmann_2017-11-04', 'Forssmann_2017-11-05', 'Hench_2017-06-15', 'Hench_2017-06-16', 'Hench_2017-06-17', 'Hench_2017-06-18', 'Lederberg_2017-12-05', 'Lederberg_2017-12-06', 'Lederberg_2017-12-07', 'Lederberg_2017-12-08', 'Lederberg_2017-12-09', 'Lederberg_2017-12-10', 'Lederberg_2017-12-11', 'Moniz_2017-05-15', 'Moniz_2017-05-16', 'Moniz_2017-05-18', 'Muller_2017-01-07', 'Muller_2017-01-08', 'Muller_2017-01-09', 'Radnitz_2017-01-08', 'Radnitz_2017-01-09', 'Radnitz_2017-01-10', 'Radnitz_2017-01-11', 'Radnitz_2017-01-12', 'Richards_2017-10-29', 'Richards_2017-10-30', 'Richards_2017-10-31', 'Richards_2017-11-01', 'Richards_2017-11-02', 'Tatum_2017-12-06', 'Tatum_2017-12-07', 'Tatum_2017-12-08', 'Tatum_2017-12-09', 'Theiler_2017-10-11']


In [3]:
def time_to_sample( time ):
    return round(time * big_lfp_ts[1,0]/big_lfp_ts[1,1])

def samples( trial, intervals, dt ):
    samples = np.array([intervals[trial,0], intervals[trial,1] ])/dt
    samples = samples.astype(int)
    return samples

In [4]:
session = Sessions[1]
filename = glob.glob(rootPath+session+"_lfp/"+session+"*")[0]
print("Reading: ", filename)
with open(filename, 'r') as f:
    big_lfp = np.fromfile(f, np.int16).reshape((-1, 385)).T

# Get Timestamps and brain region info
dataPath = rootPath + "allData/"

big_lfp_ts = np.load( glob.glob(dataPath+session+"/"+session+"*.imec.lf.timestamps.npy")[0], allow_pickle=True)

rawRow = np.load(glob.glob(dataPath+session+"/channels.rawRow.npy")[0], allow_pickle=True)
intervals = np.load(glob.glob(dataPath+session+"/trials.intervals.npy")[0], allow_pickle=True)
visualTime = np.load(glob.glob(dataPath+session+"/trials.visualStim_times.npy")[0], allow_pickle=True)
df = pd.read_csv(glob.glob(dataPath+session+"/channels.brainLocation.tsv")[0], delimiter="\t")

brainLocation=df.to_numpy()
del df

Reading:  /run/media/h4wke/Elements/LFP/Cori_2016-12-17_lfp/Cori_2016-12-17_LM_g0_t0.imec.lf.bin


In [5]:
# # print(big_lfp_ts)

# # for session in Sessions:
# session= Sessions[0]
# big_lfp_ts = np.load( glob.glob(dataPath+session+"/"+session+"*.imec.lf.timestamps.npy")[0], allow_pickle=True)

# rawRow = np.load(glob.glob(dataPath+session+"/channels.rawRow.npy")[0], allow_pickle=True)
# intervals = np.load(glob.glob(dataPath+session+"/trials.intervals.npy")[0], allow_pickle=True)
# gocue = np.load(glob.glob(dataPath+session+"/trials.goCue_times.npy")[0], allow_pickle=True)
# feedback = np.load(glob.glob(dataPath+session+"/trials.feedback_times.npy")[0], allow_pickle=True)
# visualTime = np.load(glob.glob(dataPath+session+"/trials.visualStim_times.npy")[0], allow_pickle=True)
# included = np.load(glob.glob(dataPath+session+"/trials.included.npy")[0], allow_pickle=True)
# maxi = 0
# count = 0
# for i in range( len( intervals ) ):
#     ii = intervals[i]
#     gap = feedback[i]-visualTime[i]
#     if( gap > 2.5 ):
#         print(i, gap)
#     maxi = max( maxi, gap )
# #     print(ii[0], visualTime[i], gocue[i], feedback[i], ii[1], feedback[i]-visualTime[i])
#     count+=1
    
# print( maxi, count )

- Ref: https://github.com/nsteinme/steinmetz-et-al-2019/wiki/data-files for more details on the dataset descriptions

In [6]:
# Average over all the probes in each region to get a one region to one signal mapping
regions = {}
for i in range(len(brainLocation)):
    row = brainLocation[i]
    if(row[3] == 'root'):
        continue
    if row[3] not in regions:
        regions[row[3]] = { "lfp": np.copy(big_lfp[rawRow[i][0]]), "count": 1, "probes": [i]}
    else:
        regions[row[3]]["lfp"] = regions[row[3]]["lfp"] + big_lfp[rawRow[i][0]]
        regions[row[3]]["count"] += 1
        regions[row[3]]["probes"].append(i)

for r in regions:
    regions[r]["lfp"] = regions[r]["lfp"]/regions[r]["count"]
    regions[r]["lfp"] = signal.detrend( regions[r]["lfp"], type="constant")

### Todo:
- line noise filtering

In [7]:
fs = 2500
dt = 1/fs
t_trial = 2.5
n_samples = int( t_trial*fs )
export = {'brain_area_lfp': [], 'lfp': np.zeros([len(regions),len(intervals),n_samples])}
i = 0
region_names = list(regions.keys())
for ri in range(len(region_names)):
    region = region_names[ri]
    regions[region]["trials"] = []
    export['brain_area_lfp'].append( region )
#     export['lfp'].append( [] )
    for i in range(len(intervals)):
        start = time_to_sample( visualTime[i][0] )
#         print(regions[region]["lfp"].shape)
        trial = regions[region]["lfp"][start:start+n_samples ]
#         print(len(trial))
#         regions[ri]["trials"].append(trial)
        export['lfp'][ri,i] = trial
#     print(regions[ri]["trials"])
        
#         regions[]

In [8]:
# @Title Smoothing and bandpass filters

def butter_bandpass(lowcut, highcut, fs, order=5):
    low = lowcut
    high = highcut
    sos = butter(order, [low, high], btype='bandpass', output='sos', fs=fs)
    return sos


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5, axis=-1):
    sos = butter_bandpass(lowcut, highcut, fs, order=order)
    y = sosfiltfilt(sos, data, axis=axis)
    return y


In [9]:
# from scipy import signal

# from scipy.fft import fftshift

# fs = 2500
# dt = 1/fs

# samples = np.array([intervals[0,0], intervals[-1,1] ])/dt
# samples = samples.astype(int)


# fs = 2500
# sig = regions['MEA']["lfp"]
# rec = regions[ri]["trials"][0]
# after = sig[samples[1]:]
# # hp = butter_bandpass_filter(sig, 1, 500, 2500)
# f1 = 5
# f2 = 5
# down = signal.decimate(sig, f1)
# down_2 = signal.decimate(down, f2)

# f, ax = plt.subplots(5,1, figsize=(14,12))
# # spec = plt.specgram(hp[0:250000], Fs = 2500)
# ax[1].specgram(sig, Fs = fs, cmap='rainbow')
# ax[2].specgram(down_2, Fs = fs/(f1*f2), cmap='rainbow')
# ax[0].psd(sig, Fs=2500)
# ax[3].psd(rec, Fs=2500)
# ax[4].psd(after, Fs=2500)

# f, t, Sxx = signal.spectrogram(sig[0:25000], fs)

# plt.pcolormesh(t, f, Sxx)

# plt.ylabel('Frequency [Hz]')

# plt.xlabel('Time [sec]')

# spec = plt.specgram(down_2, Fs = fs/(f1*f2), cmap='rainbow')
# plt.ylabel('Frequency [Hz]')
# plt.xlabel('Time [sec]')
# plt.show()
# down.shape


# x = np.tile(spec[1],(len(spec[2]), 1)).T
# y = np.tile(spec[2],(len(spec[1]), 1)) # transpose
# z = spec[0]

# fig = plt.figure(figsize=(14, 12))
# ax = plt.axes(projection='3d')    

# ax.plot_surface(x, y, z,cmap='rainbow', edgecolor='none')
# ax.set_title('Surface plot')
# ax.set_zlim(0,100)
# plt.show()

In [10]:
# from scipy.fftpack import fft

# dt = 1/fs
# samples = intervals[0]/dt
# samples = samples.astype(int)
# raw = regions['MEA']["lfp"][samples[0]:samples[1]]
# N=len(raw)

# f, ax = plt.subplots(6, 1, figsize=(14, 12), sharex=True)

# alpha = butter_bandpass_filter( raw, 8, 13, fs )
# theta = butter_bandpass_filter( raw, 4, 8, fs )
# beta = butter_bandpass_filter( raw, 13, 32, fs )
# gamma = butter_bandpass_filter( raw, 32, 100, fs )
# delta = butter_bandpass_filter( raw, 0.4, 4, fs )
# ax[5].plot(raw, color='k', label='raw signal' )
# ax[2].plot(alpha, color='g', label='alpha signal')
# ax[3].plot(beta, color='r', label='beta signal')
# ax[4].plot(gamma, color='b', label='gamma signal')
# ax[1].plot(theta, color='brown', label='theta signal')
# ax[0].plot(delta, color='y', label='delta signal')
# ax[0].title.set_text('(a)')
# ax[1].title.set_text('(b)')
# ax[2].title.set_text('(c)')
# ax[3].title.set_text('(d)')
# ax[4].title.set_text('(e)')
# ax[5].title.set_text('(f)')



# plt.tight_layout()


In [11]:
# with np.printoptions( suppress=True):
#     print( big_lfp_ts )
export_folder = "/run/media/h4wke/Elements/LFP/big_lfp/"
np.save(export_folder+session+".lfp.processed.npy", export, allow_pickle=True)



#TODO:
- Split into trials
- Do Specgram of 1 trial
- Try running normal analysis of that trial set